# Deep Q-Learning for Lunar Landing

## Part 1 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!pip install swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


### Importing the libraries

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 2 - Building the AI

### Creating the architecture of the Neural Network

In [ ]:
class Brain(nn.Module):

  def __init__(self, state_size, action_size, seed = 42):
      super(Brain,self).__init__()
      self.seed = torch.manual_seed(seed)
      self.fc1 = nn.Linear(state_size, 64)
      self.fc2 = nn.Linear(64, 64)
      self.fc3 = nn.Linear(64, action_size)

  def forward(self, state):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    return x

## Part 3 - Training the AI

### Setting up the environment

In [ ]:
import gymnasium as gym
env = gym.make('LunarLander-v3')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
action_shape = env.action_space.n
print('State shape',state_shape)
print('State size',state_size)
print('Action shape',action_shape)

State shape (8,)
State size 8
Action shape 4


### Initializing the hyperparameters

In [ ]:
learning_rate = 5e-4
minibatch_size = 100
discount_factor = 0.99
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3

### Implementing Experience Replay

In [ ]:
class ReplayMemory(object):

  def __init__(self, capacity):
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []

  def push(self,event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  def sample(self, batch_size):
    experiences = random.sample(self.memory, k = batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return (states, actions, rewards, next_states, dones)


### Implementing the DQN class

In [ ]:
class Agent():

  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    self.local_network = Brain(state_size, action_size).to(self.device)
    self.target_network = Brain(state_size, action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_network.parameters(), lr = learning_rate)
    self.memory = ReplayMemory(replay_buffer_size)
    self.time_step = 0

  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.time_step = (self.time_step + 1) % 4
    if self.time_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)


  def act(self, state, epsilion = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_network.eval()
    with torch.no_grad():
      action_values = self.local_network(state)
    self.local_network.train()
    if random.random() > epsilion:
      return np.argmax(action_values.cpu().data.numpy())
    else :
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, actions, rewards, next_states, dones = experiences
    next_q_targets = self.target_network(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + (discount_factor * next_q_targets * (1 - dones))
    q_expected = self.local_network(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_network, self.target_network, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

### Initializing the DQN agent

In [ ]:
agent = Agent(state_size, action_shape)

### Training the DQN agent

In [ ]:
from collections import deque

number_episodes = 2000
max_t = 1000
epsilon_start = 1.0
epsilon_end = 0.01
epsilon_decay = 0.995
epsilon = epsilon_start
scores = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(max_t):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores.append(score)
  epsilon = max(epsilon_end, epsilon_decay * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores)), end="")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores)))
  if np.mean(scores) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode , np.mean(scores)))
    torch.save(agent.local_network.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: -174.25
Episode 200	Average Score: -93.75
Episode 300	Average Score: -15.60
Episode 400	Average Score: 44.61
Episode 500	Average Score: 130.28
Episode 600	Average Score: 186.45
Episode 665	Average Score: 200.17
Environment solved in 665 episodes!	Average Score: 200.17


## Part 4 - Visualizing the results

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('lunar_landing.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()